In [1]:
####### 10/30/23 last update by ED ###################
#### NOTE:  This assumes you ran Step 1 first
#### NOTE:  This is only used if you want to breakdown output by drug (this will re-add all cross search duplicates = TargetOnly/Drug true by unique brandname only)
                # Remember with this applied and basic unique totals no longer exist if cross-search TargetOnly/Drug mismatch for any shared PMID

import pandas as pd
import numpy as np
import os

Grant_code=pd.read_csv('new_data/resultUQ_FULL.csv')
Search_Year=pd.read_csv('new_data/Search_ID_Years.csv')
BRAND_ID=pd.read_csv('search_term_inputs/BrandName_Linker.csv')

ID_CORE_Total_Drug=pd.read_csv('new_data/P_B_Drug.csv')
ID_CORE_Total_Target=pd.read_csv('new_data/P_B_Target.csv')
ID_CORE_Total_Target = ID_CORE_Total_Target[(ID_CORE_Total_Target != 0).all(axis=1)]
ID_CORE_Total_Drug = ID_CORE_Total_Drug[(ID_CORE_Total_Drug != 0).all(axis=1)]
ID_CORE_Total_Drug = ID_CORE_Total_Drug.rename(columns={'SearchID': 'Search_ID'})
ID_CORE_Total_Target = ID_CORE_Total_Target.rename(columns={'SearchID': 'Search_ID'})

Search_Year = Search_Year.rename(columns={'SearchID': 'Search_ID'})

BRAND_ID = BRAND_ID.rename(columns={'SearchID': 'Search_ID'})

In [2]:
BRAND_ID

,Search_ID,Brand_Name
0,33,Imbruvica
1,48,Xarelto
2,53,Januvia
3,63,Eliquis
4,98,Entresto
5,135,Jardiance
6,135,Farxiga
7,235,Stelara
8,236,Enbrel
9,237,NovoLog


In [3]:
PMID_DOWNLOADED_COUNT_Drug=ID_CORE_Total_Drug[['PMID','Search_ID','TIME_CUT']]
PMID_DOWNLOADED_COUNT_Drug=PMID_DOWNLOADED_COUNT_Drug.drop_duplicates()
PMID_DOWNLOADED_COUNT_Target=ID_CORE_Total_Target[['PMID','Search_ID','TIME_CUT']]
ID_CORE_Total_Target=ID_CORE_Total_Target.drop_duplicates()

PMID_DOWNLOADED_COUNT=pd.concat([PMID_DOWNLOADED_COUNT_Drug,PMID_DOWNLOADED_COUNT_Target])

In [4]:
PMID_DOWNLOADED_Analysis = PMID_DOWNLOADED_COUNT.groupby('Search_ID')['PMID'].nunique().reset_index()
PMID_DOWNLOADED_Analysis.columns = ['Search_ID', 'PMID_Downloaded']

In [5]:
BRAND_ID['Search_ID']=BRAND_ID['Search_ID'].astype(str)
PMID_DOWNLOADED_Analysis['Search_ID']=PMID_DOWNLOADED_Analysis['Search_ID'].astype(str)

In [6]:
def check_for_drug(x):
    if 'drug' in x.lower():
        return 'Drug'
    else:
        return 'TargetOnly'
PMID_DOWNLOADED_Analysis=PMID_DOWNLOADED_Analysis.merge(BRAND_ID, how='outer')
PMID_DOWNLOADED_Analysis['Data_Type_by_Drug'] = PMID_DOWNLOADED_Analysis['Search_ID'].apply(check_for_drug)
PMID_DOWNLOADED_Analysis=PMID_DOWNLOADED_Analysis[['Brand_Name','Data_Type_by_Drug','PMID_Downloaded']]

In [7]:
PMID_DOWNLOADED_Analysis

,Brand_Name,Data_Type_by_Drug,PMID_Downloaded
0,Imbruvica,TargetOnly,1222
1,Xarelto,TargetOnly,7229
2,Januvia,TargetOnly,1906
3,Eliquis,TargetOnly,7229
4,Entresto,TargetOnly,6150
5,Jardiance,TargetOnly,2486
6,Farxiga,TargetOnly,2486
7,Stelara,TargetOnly,18055
8,Enbrel,TargetOnly,12417
9,NovoLog,TargetOnly,18955


In [8]:
ID_CORE_Total_Drug=ID_CORE_Total_Drug[['PMID','Search_ID']]
ID_CORE_Total_Drug=ID_CORE_Total_Drug.drop_duplicates()


ID_CORE_Total_Target=ID_CORE_Total_Target[['PMID','Search_ID']]
ID_CORE_Total_Target=ID_CORE_Total_Target.drop_duplicates()

In [9]:
def remove_decimal_from_id(id_value):
    try:
        # Convert to float, then to int, then to string
        return str(int(float(id_value)))
    except ValueError:
        # If it's not a number, just return the original string
        return id_value
ID_CORE_Total_Drug['Search_ID'] = ID_CORE_Total_Drug['Search_ID'].apply(remove_decimal_from_id)
ID_CORE_Total_Target['Search_ID'] = ID_CORE_Total_Target['Search_ID'].apply(remove_decimal_from_id)
Search_Year['Search_ID'] = Search_Year['Search_ID'].apply(remove_decimal_from_id)

In [10]:
Grant_code_drug=ID_CORE_Total_Drug.merge(Grant_code,how='inner')
Grant_code_Target=ID_CORE_Total_Target.merge(Grant_code,how='inner')
Grant_code=Grant_code_Target.merge(Grant_code_drug,how='outer')
Grant_code=Grant_code.merge(Search_Year,how='outer')
Grant_code=Grant_code.drop_duplicates()

Grant_code['PMID'].nunique()

14463

In [11]:
Grant_code.drop('Search__ID', axis=1, inplace=True)
Grant_code.drop('Source_Search_Type', axis=1, inplace=True)
Grant_code.drop('Search_Type', axis=1, inplace=True)

In [12]:
################ UQ_COMBO (PMID|Brand) Maker
BRAND_ID['Search_ID'] = BRAND_ID['Search_ID'].apply(remove_decimal_from_id)
Grant_code=Grant_code.merge(BRAND_ID, how='outer')
Grant_code['PMID'] = Grant_code['PMID'].apply(remove_decimal_from_id)
#Grant_code['UQ_COMBO']=Grant_code['PMID'].astype(str)+"|"+Grant_code['Brand_Name']

Grant_code['UQ_COMBO_APY']=Grant_code['ACTUAL_PROJECT_YEAR'].astype(str)+"|"+Grant_code['Brand_Name']

In [13]:
################ UQ_COMBO (PMID|Brand) Maker
BRAND_ID['Search_ID'] = BRAND_ID['Search_ID'].apply(remove_decimal_from_id)
Grant_code=Grant_code.merge(BRAND_ID, how='outer')
Grant_code['PMID'] = Grant_code['PMID'].apply(remove_decimal_from_id)
#Grant_code['UQ_COMBO']=Grant_code['PMID'].astype(str)+"|"+Grant_code['Brand_Name']

Grant_code['UQ_COMBO_PMID']=Grant_code['PMID'].astype(str)+"|"+Grant_code['Brand_Name']

In [14]:
def COMBO_Linker(search_id):
    if 'drug' in search_id:
        return 'Drug'
    elif search_id.isdigit():
        return 'TargetOnly'
    else:
        return 'Error'

Grant_code['Data_Type_by_Drug'] = Grant_code['Search_ID'].apply(COMBO_Linker)

# Identify UQ_COMBO values which have at least one Drug
uq_combo_with_drug = Grant_code[Grant_code['Data_Type_by_Drug'] == 'Drug']['UQ_COMBO_APY'].unique()

# Update rows with those UQ_COMBO values to "Drug"
Grant_code.loc[Grant_code['UQ_COMBO_APY'].isin(uq_combo_with_drug), 'Data_Type_by_Drug'] = 'Drug'

In [15]:
# Identify UQ_COMBO APY values which have at least one Drug
Grant_code['Data_Type_by_APY'] = Grant_code['Search_ID'].apply(COMBO_Linker)
uq_combo_with_drug = Grant_code[Grant_code['Data_Type_by_APY'] == 'Drug']['ACTUAL_PROJECT_YEAR'].unique()

# Update rows with those UQ_COMBO values to "Drug"
Grant_code.loc[Grant_code['ACTUAL_PROJECT_YEAR'].isin(uq_combo_with_drug), 'Data_Type_by_APY'] = 'Drug'

In [16]:
# Identify UQ_COMBO PMID values which have at least one Drug
Grant_code['Data_Type_by_PMID'] = Grant_code['Search_ID'].apply(COMBO_Linker)
uq_combo_with_drug = Grant_code[Grant_code['Data_Type_by_PMID'] == 'Drug']['UQ_COMBO_PMID'].unique()

# Update rows with those UQ_COMBO values to "Drug"
Grant_code.loc[Grant_code['UQ_COMBO_PMID'].isin(uq_combo_with_drug), 'Data_Type_by_PMID'] = 'Drug'

In [17]:
Grant_code = Grant_code.dropna(subset=['PMID'])
Grant_code = Grant_code[Grant_code['PMID'].ne('') & Grant_code['PMID'].ne(0)]

In [18]:
####### TIME CUT +1 Year Correction (adjust for download year offset back to True approval cut year)
Grant_code.rename(columns={'TIME_CUT': 'TIME_CUT(Download_Used)'}, inplace=True)
Grant_code['TIME_CUT']=Grant_code['TIME_CUT(Download_Used)'].astype(int) - 1

Grant_code['TIME_CUT']=Grant_code['TIME_CUT'].apply(remove_decimal_from_id)

Grant_code['TIME_CUT']=Grant_code['TIME_CUT'].astype(str)

In [19]:
#### 2018USD inflation adjustment

Grant_code.rename(columns = {"APY": "FY"}, 
          inplace = True)

Grant_code.rename(columns = {"APY_COST_inf2018": "Original_COST"}, 
          inplace = True)

Inflation_key_V2018 = pd.read_csv('function_data/inf2018_key.csv')
core_set_2018=pd.merge(Grant_code,Inflation_key_V2018, how='outer')
core_set_2018['APY_COST_inf2018']=core_set_2018['Original_COST']*core_set_2018['inf_2018']
core_set_2018.drop('inf_2018', inplace=True, axis=1)

Grant_code=core_set_2018

Grant_code.rename(columns = {"FY": "APY"}, 
          inplace = True)

In [20]:
Grant_code = Grant_code.dropna(subset=['PMID'])
Grant_code = Grant_code[Grant_code['PMID'].ne('') & Grant_code['PMID'].ne(0)]

In [21]:
## Debug Test
# Grant_code=Grant_code[Grant_code['Brand_Name']=='Xarelto']

In [22]:
# ######## Time cut by PMID PubYear to approval year
# Grant_code['TIME_CUT_FIC'] = Grant_code['TIME_CUT']
# Grant_code.loc[Grant_code['Search_ID'] == "135", 'TIME_CUT_FIC'] = "2013"
# Grant_code.loc[Grant_code['Search_ID'] == "63", 'TIME_CUT_FIC'] = "2011"

# Grant_code_CC=Grant_code.copy()

# Grant_code['Years_from_app'] = Grant_code['APY'].astype(int) - Grant_code['TIME_CUT'].astype(int)
# Grant_code.to_csv("IRA_10Drugs_Main_Output.csv",index=None)

# Grant_code['Pre_Approval_APY'] = Grant_code['Years_from_app'] <= 0
# Grant_code = Grant_code[Grant_code['Pre_Approval_APY'] == 1]

# ####  Last year of total costs change >2% per year for 10 Drug dataset, 9_12_23
# Grant_code = Grant_code[Grant_code['Years_from_app'] >= -12]

# Grant_code.to_csv("Output_To_Approval.csv",index=None)

In [23]:
######## Time cut by PMID PubYear to approval year
Grant_code['TIME_CUT_FIC'] = Grant_code['TIME_CUT']

#### First in class for targets adjustment ( Use for Basic/Target search ID that have an earlier FIC drug approval as cut year )
Grant_code.loc[Grant_code['Search_ID'] == "135", 'TIME_CUT_FIC'] = "2013"
Grant_code.loc[Grant_code['Search_ID'] == "63", 'TIME_CUT_FIC'] = "2011"

Grant_code['Years_from_app'] = Grant_code['APY'].astype(int) - Grant_code['TIME_CUT'].astype(int)
Grant_code['Years_from_app_FIC_TARGET'] = Grant_code['APY'].astype(int) - Grant_code['TIME_CUT_FIC'].astype(int)

Grant_code['FY_START_Years_from_app_FIC_TARGET'] = Grant_code['FY_Start'].astype(int) - Grant_code['TIME_CUT_FIC'].astype(int)


Grant_code['Pre_Approval_APY'] = Grant_code['Years_from_app'] <= 0
Grant_code['Pre_Approval_APY_FIC'] = Grant_code['Years_from_app_FIC_TARGET'] <= 0

In [24]:
#### Use for data from 4+ Years after approval (not part of main method)
# Grant_code['4YEAR_RULE_1'] = Grant_code['Years_from_app_FIC_TARGET'].apply(lambda x: 1 if 4 >= x >= -12 else 0)
# Grant_code['4YEAR_RULE_2'] = Grant_code['FY_START_Years_from_app_FIC_TARGET'].apply(lambda x: 1 if 0 >= x else 0)

# Grant_code['APY_Start_PreApproval_4Year'] = np.where(Grant_code['4YEAR_RULE_1'] + Grant_code['4YEAR_RULE_2'] == 2, 1, 0)

# Grant_code.drop('4YEAR_RULE_1', axis=1, inplace=True)
# Grant_code.drop('4YEAR_RULE_2', axis=1, inplace=True)

####  Last year of total costs change >2% per year for 10 Drug dataset, 9_12_23
Grant_code['0_12_Year_Rule_FIC'] = Grant_code['Years_from_app_FIC_TARGET'].apply(lambda x: 1 if 0 >= x >= -12 else 0)
Grant_code=Grant_code.fillna(0)
Grant_code_output=Grant_code

In [25]:
cols_order = ["Brand_Name", 
"Search_ID", 
"PMID", 
"PUB_YEAR", 
"PROJECT_NUMBER", 
"FY_Start", 
"FY_Last", 
"APY", 
"ACTUAL_PROJECT_YEAR", 
"APY_COST_inf2018", 
"Original_COST", 
"Activity_Code", 
"Institute_Code", 
"Acronym_institute_name", 
"full_institute_name", 
"Compressed Names", 
"Project_Count", 
"Grant_Type_Name", 
"Data_Type_by_Drug", 
"Data_Type_by_APY", 
"Data_Type_by_PMID", 
"UQ_COMBO_APY", 
"UQ_COMBO_PMID", 
"TIME_CUT(Download_Used)", 
"TIME_CUT", 
"TIME_CUT_FIC", 
"Years_from_app", 
"Years_from_app_FIC_TARGET", 
"FY_START_Years_from_app_FIC_TARGET", 
"Pre_Approval_APY", 
"Pre_Approval_APY_FIC", 
"0_12_Year_Rule_FIC"
]
Grant_code_output = Grant_code_output[cols_order]

In [26]:
Grant_code_output.to_csv("IRA_10Drugs_Main_Output.csv",index=None)

In [27]:
Grant_code['CC_ID']= Grant_code['Brand_Name']+"_"+Grant_code['Data_Type_by_Drug']+"_"+Grant_code['ACTUAL_PROJECT_YEAR']

In [28]:
Grant_code_ID=Grant_code[['CC_ID','Data_Type_by_Drug','Brand_Name']]
Grant_code_ID=Grant_code_ID.drop_duplicates()

In [29]:
########## Cost of Cap 10.5% and 3%

In [30]:
############ USE FOR FIC CC analysis only
Grant_code_CC=Grant_code.copy()

Grant_code_CC['Years_from_app'] = Grant_code_CC['APY'].astype(int) - Grant_code_CC['TIME_CUT_FIC'].astype(int)
Grant_code_CC['Pre_Approval_APY'] = Grant_code_CC['Years_from_app'] <= 0
Grant_code_CC = Grant_code_CC[Grant_code_CC['Pre_Approval_APY'] == 1]
Grant_code_CC = Grant_code_CC[Grant_code_CC['Years_from_app'] >= -12]

Grant_code_CC=Grant_code_CC[['CC_ID','Years_from_app','Original_COST']]
Grant_code_CC=Grant_code_CC.drop_duplicates()

In [31]:
Grant_code_CC=pd.pivot_table(Grant_code_CC,index='CC_ID',columns='Years_from_app',values='Original_COST', aggfunc='sum')
Grant_code_CC=Grant_code_CC.fillna(0)

In [32]:
Grant_code_CC.to_csv('CC_CHECK.csv')

In [33]:
Grant_code_CC=pd.read_csv('CC_CHECK.csv')

In [34]:
Grant_code_CC=Grant_code_CC.merge(Grant_code_ID,how='inner')

In [35]:
Grant_code_CC.to_csv('CC_DEBUG.csv',index=None)

In [36]:
Grant_code_CC_Grouped = Grant_code_CC.groupby(['Brand_Name', 'Data_Type_by_Drug']).sum().reset_index()

In [37]:
Grant_code_CC_Grouped

,Brand_Name,Data_Type_by_Drug,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
0,Eliquis,Drug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,262484.0,2772575.0,24155262.0,10834932.0
1,Eliquis,TargetOnly,31810075.0,36668812.0,44105029.0,50332882.0,38770518.0,49277741.0,44752964.0,75059001.0,41412165.0,47308460.0,55556061.0,37067908.0,31457338.0
2,Enbrel,Drug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1454764.0,0.0,0.0
3,Enbrel,TargetOnly,22966560.0,28221251.0,58050598.0,52084521.0,100065040.0,114099206.0,110892323.0,129159398.0,161223818.0,156971308.0,155409195.0,171071995.0,250943834.0
4,Entresto,Drug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2918701.0,3433465.0,0.0
5,Entresto,TargetOnly,34719438.0,32920577.0,59803750.0,90517635.0,33550009.0,24616357.0,69349104.0,76436290.0,109916001.0,56493502.0,90320158.0,35675815.0,52220013.0
6,Farxiga,Drug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,524315.0,2907918.0,2777743.0,1316665.0,1465841.0,8178823.0
7,Farxiga,TargetOnly,15942690.0,6403843.0,12578745.0,31422662.0,17306904.0,35298954.0,12059649.0,34253940.0,37575441.0,47564809.0,45162325.0,22617015.0,35346768.0
8,Imbruvica,Drug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,752887.0,4472552.0,14044082.0,58229816.0,44656978.0
9,Imbruvica,TargetOnly,21677004.0,23562610.0,16579366.0,10156266.0,19711392.0,19056513.0,20514510.0,22039802.0,33456489.0,67738626.0,41863462.0,17724151.0,46206519.0


In [38]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


year_cols = [col for col in Grant_code_CC_Grouped.columns if is_number(col)]
sorted_cols = sorted(year_cols, key=float, reverse=True)  # Sorts 0, -1, -2...

for col in sorted_cols:
    Grant_code_CC_Grouped[col] = Grant_code_CC_Grouped[col].astype(int)
Grant_code_CC_Grouped['Discount_Type']= 'Original'
final_cols = ['Brand_Name', 'Data_Type_by_Drug','Discount_Type'] + sorted_cols
Grant_code_CC_Grouped = Grant_code_CC_Grouped[final_cols]

#Grant_code_CC_Grouped.to_csv('CC_10Drugs_Raw.csv',index=None)
Grant_code_CC_Grouped_Raw=Grant_code_CC_Grouped.copy()

In [39]:
def modify_rightmost_column(df):
    # Identify the rightmost column
    rightmost_col = df.columns[-1]
    
    # Multiply all values in the rightmost column by 1 (Hand analysis for 10.5% or 3% as needed)
    df[rightmost_col] *= 1
    
    return df

df_modified = modify_rightmost_column(Grant_code_CC_Grouped)
df_modified

,Brand_Name,Data_Type_by_Drug,Discount_Type,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12
0,Eliquis,Drug,Original,10834932,24155262,2772575,262484,0,0,0,0,0,0,0,0,0
1,Eliquis,TargetOnly,Original,31457338,37067908,55556061,47308460,41412165,75059001,44752964,49277741,38770518,50332882,44105029,36668812,31810075
2,Enbrel,Drug,Original,0,0,1454764,0,0,0,0,0,0,0,0,0,0
3,Enbrel,TargetOnly,Original,250943834,171071995,155409195,156971308,161223818,129159398,110892323,114099206,100065040,52084521,58050598,28221251,22966560
4,Entresto,Drug,Original,0,3433465,2918701,0,0,0,0,0,0,0,0,0,0
5,Entresto,TargetOnly,Original,52220013,35675815,90320158,56493502,109916001,76436290,69349104,24616357,33550009,90517635,59803750,32920577,34719438
6,Farxiga,Drug,Original,8178823,1465841,1316665,2777743,2907918,524315,0,0,0,0,0,0,0
7,Farxiga,TargetOnly,Original,35346768,22617015,45162325,47564809,37575441,34253940,12059649,35298954,17306904,31422662,12578745,6403843,15942690
8,Imbruvica,Drug,Original,44656978,58229816,14044082,4472552,752887,0,0,0,0,0,0,0,0
9,Imbruvica,TargetOnly,Original,46206519,17724151,41863462,67738626,33456489,22039802,20514510,19056513,19711392,10156266,16579366,23562610,21677004


In [40]:
import pandas as pd

def modify_adjacent_column(df, col_name):
    # Identify the specified column and the one to its left
    adjacent_col = df.columns[df.columns.get_loc(col_name) - 1]
    
    # Check if both columns are numeric
    if pd.api.types.is_numeric_dtype(df[col_name]) and pd.api.types.is_numeric_dtype(df[adjacent_col]):
        # For each row, add the value from the specified column to the value in the adjacent column, then multiply by 1.105
        df[adjacent_col] = (df[adjacent_col] + df[col_name]) * 1
    
    return df

def modify_until_discount(df):
    # Begin with the column left of the rightmost column and move leftward until "Discount_Type"
    col_index = len(df.columns) - 2  # Starting one column left of the rightmost column
    while col_index > 0:  # Safety to ensure we don't go out of bounds
        if df.columns[col_index] == "Discount_Type":
            break
        df = modify_adjacent_column(df, df.columns[col_index])
        col_index -= 1

    return df

df_modified = modify_until_discount(df_modified)

In [41]:
df_modified['Discount_Type'] ="0%"

In [42]:
df_modified=pd.concat([df_modified, Grant_code_CC_Grouped_Raw])

In [43]:
df_modified.sort_values(by='Brand_Name')
df_modified.sort_values(by=['Discount_Type', 'Brand_Name'])
df_modified.to_csv('CC_10Drugs_0%_HandCheck.csv',index=None)

In [44]:
Grant_code

,PMID,Search_ID,PUB_YEAR,PROJECT_NUMBER,FY_Start,FY_Last,APY,ACTUAL_PROJECT_YEAR,Original_COST,Activity_Code,...,TIME_CUT,APY_COST_inf2018,TIME_CUT_FIC,Years_from_app,Years_from_app_FIC_TARGET,FY_START_Years_from_app_FIC_TARGET,Pre_Approval_APY,Pre_Approval_APY_FIC,0_12_Year_Rule_FIC,CC_ID
0,2865955,53,1985.0,R01NS017392,1985.0,2001.0,1985.0,1985R01NS017392,122880.0,R01,...,2006,2.867651e+05,2006,-21,-21,-21,True,True,0,Januvia_TargetOnly_1985R01NS017392
1,2865955,53,1985.0,T32GM007280,1985.0,2020.0,1985.0,1985T32GM007280,316529.0,T32,...,2006,7.386837e+05,2006,-21,-21,-21,True,True,0,Januvia_TargetOnly_1985T32GM007280
2,2864346,53,1985.0,R01GM029185,1985.0,1998.0,1985.0,1985R01GM029185,258444.0,R01,...,2006,6.031308e+05,2006,-21,-21,-21,True,True,0,Januvia_TargetOnly_1985R01GM029185
3,2864346,53,1985.0,R23AM034138,1985.0,1985.0,1985.0,1985R23AM034138,54096.0,R23,...,2006,1.262438e+05,2006,-21,-21,-21,True,True,0,Januvia_TargetOnly_1985R23AM034138
4,2582028,53,1985.0,R01AM025578,1985.0,1985.0,1985.0,1985R01AM025578,88640.0,R01,...,2006,2.068592e+05,2006,-21,-21,-21,True,True,0,Januvia_TargetOnly_1985R01AM025578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30042,27872097,98,2017.0,R01CA178856,2014.0,2018.0,2017.0,2017R01CA178856,340100.0,R01,...,2015,3.483984e+05,2015,2,2,-1,False,False,0,Entresto_TargetOnly_2017R01CA178856
30043,27872097,98,2017.0,P30DK050306,1997.0,2020.0,2017.0,2017P30DK050306,1384555.0,P30,...,2015,1.418338e+06,2015,2,2,-18,False,False,0,Entresto_TargetOnly_2017P30DK050306
30044,27872097,98,2017.0,P30CA010815,1985.0,2021.0,2017.0,2017P30CA010815,2856108.0,P30,...,2015,2.925797e+06,2015,2,2,-30,False,False,0,Entresto_TargetOnly_2017P30CA010815
30045,27868321,98,2017.0,R01HL123478,2015.0,2018.0,2017.0,2017R01HL123478,384303.0,R01,...,2015,3.936800e+05,2015,2,2,0,False,False,0,Entresto_Drug_2017R01HL123478


In [45]:
Grant_code_FIC_RULE = Grant_code.loc[Grant_code['0_12_Year_Rule_FIC'] == 1]

In [46]:
Grant_code_unique_COST = Grant_code_FIC_RULE.drop_duplicates(subset=['Brand_Name', 'Data_Type_by_Drug', 'ACTUAL_PROJECT_YEAR', 'APY_COST_inf2018'])

result_COST = Grant_code_unique_COST.groupby(['Brand_Name', 'Data_Type_by_Drug'])['APY_COST_inf2018'].sum().reset_index()

result_COST = result_COST.rename(columns={'APY_COST_inf2018':'COST_SUM'})

In [47]:
Grant_code_unique_APY=Grant_code_FIC_RULE[['Brand_Name', 'Data_Type_by_Drug', 'ACTUAL_PROJECT_YEAR']]
Grant_code_unique_APY=Grant_code_unique_APY.drop_duplicates()

result_APY = Grant_code_unique_APY.groupby(['Brand_Name', 'Data_Type_by_Drug'])['ACTUAL_PROJECT_YEAR'].nunique().reset_index()
result_APY.rename(columns={'ACTUAL_PROJECT_YEAR': 'Count_of_APY(0-12Year)'}, inplace=True)

In [48]:
Grant_code_unique_PMID=Grant_code_FIC_RULE[['Brand_Name', 'Data_Type_by_PMID', 'PMID']]
Grant_code_unique_PMID=Grant_code_unique_PMID.drop_duplicates()


Grant_code_unique_PMID.rename(columns={'Data_Type_by_PMID': 'Data_Type_by_Drug'}, inplace=True)
result_PMID = Grant_code_unique_PMID.groupby(['Brand_Name', 'Data_Type_by_Drug'])['PMID'].nunique().reset_index()
result_PMID.rename(columns={'PMID': 'NIH_PMID(0_12Year)'}, inplace=True)


result_Items = result_PMID.merge(result_APY, on=['Brand_Name', 'Data_Type_by_Drug'], how='left')
Item_Cost_Overview_12_YEARS = result_Items.merge(result_COST, on=['Brand_Name', 'Data_Type_by_Drug'], how='left')

Item_Cost_Overview_12_YEARS['COST_SUM_Mills']=Item_Cost_Overview_12_YEARS['COST_SUM']/1000000
Item_Cost_Overview_12_YEARS.drop('COST_SUM', axis=1, inplace=True)

In [49]:
Grant_code_unique_COST = Grant_code.drop_duplicates(subset=['Brand_Name', 'Data_Type_by_Drug', 'ACTUAL_PROJECT_YEAR', 'APY_COST_inf2018'])

result_COST = Grant_code_unique_COST.groupby(['Brand_Name', 'Data_Type_by_Drug'])['APY_COST_inf2018'].sum().reset_index()

result_COST = result_COST.rename(columns={'APY_COST_inf2018':'COST_SUM'})

Grant_code_unique_APY=Grant_code[['Brand_Name', 'Data_Type_by_Drug', 'ACTUAL_PROJECT_YEAR']]
Grant_code_unique_APY=Grant_code_unique_APY.drop_duplicates()

#Grant_code_unique_APY.rename(columns={'Data_Type_by_APY': 'Data_Type_by_Drug'}, inplace=True)

result_APY = Grant_code_unique_APY.groupby(['Brand_Name', 'Data_Type_by_Drug'])['ACTUAL_PROJECT_YEAR'].nunique().reset_index()
result_APY.rename(columns={'ACTUAL_PROJECT_YEAR': 'Count_of_APY(ALL_YEAR)'}, inplace=True)

Grant_code_unique_PMID=Grant_code[['Brand_Name', 'Data_Type_by_PMID', 'PMID']]
Grant_code_unique_PMID=Grant_code_unique_PMID.drop_duplicates()


Grant_code_unique_PMID.rename(columns={'Data_Type_by_PMID': 'Data_Type_by_Drug'}, inplace=True)
result_PMID = Grant_code_unique_PMID.groupby(['Brand_Name', 'Data_Type_by_Drug'])['PMID'].nunique().reset_index()
result_PMID.rename(columns={'PMID': 'NIH_PMID(ALL_YEAR)'}, inplace=True)


result_Items = result_PMID.merge(result_APY, on=['Brand_Name', 'Data_Type_by_Drug'], how='left')
Item_Cost_Overview_ALL_YEARS = result_Items.merge(result_COST, on=['Brand_Name', 'Data_Type_by_Drug'], how='left')

Item_Cost_Overview_ALL_YEARS['COST_SUM_Mills(ALL_YEAR)']=Item_Cost_Overview_ALL_YEARS['COST_SUM']/1000000
Item_Cost_Overview_ALL_YEARS.drop('COST_SUM', axis=1, inplace=True)

In [50]:
Item_Cost_Overview_ALL = Item_Cost_Overview_12_YEARS.merge(Item_Cost_Overview_ALL_YEARS, on=['Brand_Name', 'Data_Type_by_Drug'], how='inner')
Item_Cost_Overview_ALL = PMID_DOWNLOADED_Analysis.merge(Item_Cost_Overview_ALL, on=['Brand_Name', 'Data_Type_by_Drug'], how='inner')

In [51]:
Item_Cost_Overview_ALL

,Brand_Name,Data_Type_by_Drug,PMID_Downloaded,NIH_PMID(0_12Year),Count_of_APY(0-12Year),COST_SUM_Mills,NIH_PMID(ALL_YEAR),Count_of_APY(ALL_YEAR),COST_SUM_Mills(ALL_YEAR)
0,Imbruvica,TargetOnly,1222,195,369,432.474660,317,574,665.249866
1,Xarelto,TargetOnly,7229,575,702,745.461454,1234,1469,1423.686133
2,Januvia,TargetOnly,1906,154,213,227.253202,237,334,322.755676
3,Eliquis,TargetOnly,7229,577,705,751.822553,1237,1474,1431.526204
4,Entresto,TargetOnly,6150,383,689,894.332491,961,1404,1704.661000
5,Jardiance,TargetOnly,2486,256,414,428.073901,463,737,658.244927
6,Farxiga,TargetOnly,2486,252,403,423.266289,460,725,650.906320
7,Stelara,TargetOnly,18055,3683,5281,6467.122908,4219,6255,7562.478951
8,Enbrel,TargetOnly,12417,2312,3077,2603.988694,2510,3459,3091.371252
9,NovoLog,TargetOnly,18955,3660,3940,3106.785168,4527,4862,4064.475930


In [52]:
Item_Cost_Overview_ALL['%_PMID_Funded(0_12)']=Item_Cost_Overview_ALL['NIH_PMID(0_12Year)']/Item_Cost_Overview_ALL['PMID_Downloaded']
Item_Cost_Overview_ALL['%_PMID_Funded(ALL_YEAR)']=Item_Cost_Overview_ALL['NIH_PMID(ALL_YEAR)']/Item_Cost_Overview_ALL['PMID_Downloaded']

In [53]:
Item_Cost_Overview_ALL

,Brand_Name,Data_Type_by_Drug,PMID_Downloaded,NIH_PMID(0_12Year),Count_of_APY(0-12Year),COST_SUM_Mills,NIH_PMID(ALL_YEAR),Count_of_APY(ALL_YEAR),COST_SUM_Mills(ALL_YEAR),%_PMID_Funded(0_12),%_PMID_Funded(ALL_YEAR)
0,Imbruvica,TargetOnly,1222,195,369,432.474660,317,574,665.249866,0.159574,0.259411
1,Xarelto,TargetOnly,7229,575,702,745.461454,1234,1469,1423.686133,0.079541,0.170701
2,Januvia,TargetOnly,1906,154,213,227.253202,237,334,322.755676,0.080797,0.124344
3,Eliquis,TargetOnly,7229,577,705,751.822553,1237,1474,1431.526204,0.079817,0.171116
4,Entresto,TargetOnly,6150,383,689,894.332491,961,1404,1704.661000,0.062276,0.156260
5,Jardiance,TargetOnly,2486,256,414,428.073901,463,737,658.244927,0.102977,0.186243
6,Farxiga,TargetOnly,2486,252,403,423.266289,460,725,650.906320,0.101368,0.185036
7,Stelara,TargetOnly,18055,3683,5281,6467.122908,4219,6255,7562.478951,0.203988,0.233675
8,Enbrel,TargetOnly,12417,2312,3077,2603.988694,2510,3459,3091.371252,0.186196,0.202142
9,NovoLog,TargetOnly,18955,3660,3940,3106.785168,4527,4862,4064.475930,0.193089,0.238829


In [54]:
Item_Cost_Overview_ALL.to_csv('Item_Cost_Overview.csv',index=None)

In [55]:
os.remove('CC_CHECK.csv')